In [ ]:
import cv2
import numpy as np
import os
from skimage.filters import hessian
from skimage import io
import matplotlib.pyplot as plt
from scipy import ndimage as nd
from sklearn.cluster import KMeans


In [ ]:
def calculate_ridge_density(image):
    # Step 1: Preprocess the image with consistent parameters
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)
    _, binary = cv2.threshold(edges, 30, 255, cv2.THRESH_BINARY)

    # Step 2: Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 3: Calculate ridge density
    total_area = binary.shape[0] * binary.shape[1]
    ridge_length = 0

    # Set a minimum contour length to filter out small contours
    min_contour_length = 20  # Adjust this threshold as needed

    for contour in contours:
        length = cv2.arcLength(contour, closed=False)
        if length > min_contour_length:
            ridge_length += length

    ridge_density = ridge_length / total_area

    return ridge_density


In [ ]:
def calculate_bead_arrangement_distance(image_path):
    # Load the image
    image = cv2.imread(image_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform edge detection
    edges = cv2.Canny(gray, 40, 40, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    bead_distances = []
    for contour in contours:
        # Calculate the center of mass (centroid) of each contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            bead_distances.append((cX, cY))

    if len(bead_distances) > 1:
        total_distance = 0
        for i in range(len(bead_distances) - 1):
            x1, y1 = bead_distances[i]
            x2, y2 = bead_distances[i + 1]
            distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            total_distance += distance
        average_distance = total_distance / (len(bead_distances) - 1)
    else:

        average_distance = 0.0

    # Scaling and Mapping (0-9)
    min_distance = 0.0
    max_distance = 100.0
    mapped_value = 0 + (average_distance - min_distance) * (9 - 0) / (max_distance - min_distance)

    # Return the mapped value as an integer
    return int(mapped_value)

In [ ]:


def calculate_average_inter_ridge_distance(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Preprocess the image (e.g., apply Gaussian blur and thresholding)
    #image = cv2.GaussianBlur(image, (5, 5), 0)
    _, threshed = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Detect the ridge pattern using Hough Line Transform
    lines = cv2.HoughLinesP(threshed, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

    # Initialize a list to store inter-ridge distances
    inter_ridge_distances = []

    # Calculate inter-ridge distances
    for line1 in lines:
        x1, y1, x2, y2 = line1[0]
        for line2 in lines:
            x3, y3, x4, y4 = line2[0]
            distance = np.sqrt((x1 - x3) ** 2 + (y1 - y3) ** 2)
            inter_ridge_distances.append(distance)

    # Calculate the average inter-ridge distance
    average_distance = np.mean(inter_ridge_distances)

    return average_distance

# Example of how to use the function:
#image_path = "imgs\\iota-h\\hessian_image.png"
#average_distance = calculate_average_inter_ridge_distance(image_path)
#print("Average Inter-Ridge Distance:", average_distance)


In [ ]:




def detect_beads(muzzle_region):
    gray_muzzle = cv2.cvtColor(muzzle_region, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, binary_muzzle = cv2.threshold(gray_muzzle, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_muzzle, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out small contours (adjust this threshold as needed)
    min_contour_area = 100  # Minimum contour area to consider as a bead
    beads = [contour for contour in contours if cv2.contourArea(contour) >= min_contour_area]

    return beads


def estimate_bead_size(bead):
    # Calculate the area of the bead contour
    bead_area = cv2.contourArea(bead)
    return bead_area


  # Provide appropriate units

In [ ]:
def average_bead_size(image):


    beads = detect_beads(image)

    bead_sizes = [estimate_bead_size(bead) for bead in beads]

    average_size = np.mean(bead_sizes)

    return average_size


In [ ]:
from skimage.feature import local_binary_pattern


def calculate_ridge_orientation(image_path, analyze_left_side=True, num_orientations=16):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Determine the region of interest (left side or right side)
    if analyze_left_side:
        roi = image[:, :image.shape[1] // 2]
    else:
        roi = image[:, image.shape[1] // 2:]

    def ridge_orientation_lbp(roi, radius=1, neighbors=8, block_size=16):
        # Calculate LBP image
        lbp_image = local_binary_pattern(roi, neighbors, radius, method='uniform')

        # Calculate gradient of the LBP image
        gradient_x = cv2.Sobel(lbp_image, cv2.CV_64F, 1, 0, ksize=3)
        gradient_y = cv2.Sobel(lbp_image, cv2.CV_64F, 0, 1, ksize=3)

        # Calculate the structure tensor based on LBP gradient
        gradient_xx = gradient_x * gradient_x
        gradient_yy = gradient_y * gradient_y
        gradient_xy = gradient_x * gradient_y

        # Calculate ridge density
        ridge_density = gradient_xx + gradient_yy

        # Apply Gaussian weighting to the structure tensor based on ridge density
        kernel = cv2.getGaussianKernel(block_size, -1)
        kernel = kernel.dot(kernel.T)

        weighted_xx = cv2.filter2D(gradient_xx, -1, kernel)
        weighted_yy = cv2.filter2D(gradient_yy, -1, kernel)
        weighted_xy = cv2.filter2D(gradient_xy, -1, kernel)

        # Calculate ridge orientation
        ridge_orientation = 0.5 * np.arctan2(2 * weighted_xy, weighted_xx - weighted_yy)

        return ridge_orientation

    # Calculate ridge orientation for the selected region
    ridge_orientation = ridge_orientation_lbp(roi)

    # Convert ridge orientation from radians to degrees
    ridge_orientation_degrees = np.degrees(ridge_orientation)

    # Restrict ridge orientations to the range 0-180 degrees
    ridge_orientation_degrees = (ridge_orientation_degrees + 180) % 180

    # Split the range into multiple bins to consider multiple orientations
    bin_size = 180 / num_orientations
    orientation_histogram = np.histogram(ridge_orientation_degrees, bins=num_orientations, range=(0, 180))[0]

    # Find the consensus orientation
    consensus_orientation = np.argmax(orientation_histogram) * bin_size

    return float(consensus_orientation)

# Example usage to analyze the left side of the image and consider 8 orientations
#image_path = "imgs\Real\cattle_1500_DSCF7159.jpg"
#ridge_orientation = calculate_ridge_orientation(image_path, analyze_left_side=True, num_orientations=8)
#print("Consensus Ridge Orientation (Degrees, 0-180):", ridge_orientation)

In [ ]:


# Define the directory containing your dataset
#dataset_dir = 'imgs/'

# Function to extract bead density from an image
def extract_bead_density(image):
    # Binarization using adaptive thresholding
    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Connected component analysis to label beads
    num_labels, labeled_image = cv2.connectedComponents(binary_image)

    # Calculate the total area of beads
    total_bead_area = np.sum(binary_image == 255)

    # Calculate the bead density
    bead_density = num_labels / total_bead_area

    return bead_density





# Print or use the bead density values for each image






In [ ]:
image_directory = "/content/iota-h"


feature_values = []
density_values = []
orientation_values = []
ridge_density = []
average_beadsize = []
#average_IR_distance = []

# Loop through all files in the directory
for filename in os.listdir(image_directory):


    image_path = os.path.join(image_directory, filename)
    image = cv2.imread(image_path)
    new_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


    feature_value = calculate_bead_arrangement_distance(image_path)


    bead_density = round(extract_bead_density(new_image)*10e5)
    orientation_value = calculate_ridge_orientation(image_path, analyze_left_side=True, num_orientations=16)
    #imagerg =  cv2.cvtColor(image_path, cv2.COLOR_BGR2GRAY)
    ridge_dense = round(calculate_ridge_density(image)*10e4)
    beadsize = round(average_bead_size(image))*10e-2
    #average_distance = calculate_average_inter_ridge_distance(image_path)


    feature_values.append(feature_value)
    density_values.append(bead_density)
    orientation_values.append(orientation_value)
    ridge_density.append(ridge_dense)
    average_beadsize.append(beadsize)
   # average_IR_distance.append(average_distance)

print(feature_values,"\n")

print(density_values,"\n")
print(orientation_values,"\n")
print(ridge_density,"\n")
print(average_beadsize,"\n")
#print(average_IR_distance,"\n")


[20, 19, 17, 17, 21, 17, 21, 22, 18, 18, 20, 18] 

[15434, 44328, 28883, 13742, 8613, 44055, 4066, 8830, 44148, 40716, 11958, 24233] 

[0.0, 33.75, 0.0, 90.0, 78.75, 78.75, 90.0, 90.0, 90.0, 90.0, 90.0, 168.75] 

[21588, 33120, 30509, 17840, 14678, 32499, 13064, 17749, 32995, 33348, 15680, 32531] 

[98.7, 29.6, 74.7, 103.9, 62.2, 33.800000000000004, 100.5, 123.2, 25.900000000000002, 32.300000000000004, 38.1, 67.4] 



In [ ]:
image_directory = "/content/iota-h"

average_IR_distance = []

# Loop through all files in the directory
for filename in os.listdir(image_directory):


    image_path = os.path.join(image_directory,filename)


    average_distance = calculate_average_inter_ridge_distance(image_path)
    #new_image = cv2.imread(image_path)

    #imagerg =  cv2.cvtColor(image_path, cv2.COLOR_BGR2GRAY)
    #ridge_dense = round(calculate_ridge_density(new_image)*10e4)
   # beadsize = average_bead_size(new_image)
    average_IR_distance.append(average_distance)







print(average_IR_distance,"\n")

[425.21866076649627, 334.1518092681864, 292.36211223994263, 371.75563171263474, 356.66666970669047, 386.46492037808383, 346.6784008713963, 379.2223253595604, 369.87407404728714, 354.77376400171937, 431.0465358220984, 323.40119704228107] 



In [ ]:
def kMeans1(feature):
    # Calculate the sum of squared distances (inertia) for different values of k
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(feature.reshape(-1, 1))
        inertia.append(kmeans.inertia_)

    # Calculate the first and second derivatives of the inertia
    inertia = np.array(inertia)
    first_derivative = np.diff(inertia)
    second_derivative = np.diff(first_derivative)

    # Find the "elbow" point where the second derivative changes from negative to positive
    elbow_point = np.where(second_derivative > 0)[0][0] + 2  # Add 2 to account for the two differences calculated

    # Plot the inertia values
    # plt.plot(range(1, 8), inertia, marker='o', linestyle='-', color='b')
    # plt.xlabel('Number of clusters')
    # plt.ylabel('Inertia')
    # plt.title('Elbow Method')
    # plt.show()

    # print("Optimal number of clusters:", elbow_point + 1)

    optimal_k = elbow_point + 1  # You can determine this value visually or programmatically

    k_value1 = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    k_value1.fit(feature.reshape(-1, 1))

    # Assign clusters to data points
    labels = kmeans.predict(feature.reshape(-1, 1))

    # Visualize the clustering results
    # for i in range(optimal_k):
    #     cluster_points = feature[labels == i]
    #     plt.scatter(cluster_points, [i] * len(cluster_points), label=f'Cluster {i + 1}')

    # plt.title('K-Means Clustering')
    # plt.xlabel('Data Points')
    # plt.ylabel('Cluster')
    # plt.legend()
    # plt.show()

    return k_value1


In [ ]:
def kMeans2(feature):
    # Calculate the sum of squared distances (inertia) for different values of k
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(feature.reshape(-1, 1))
        inertia.append(kmeans.inertia_)

    # Calculate the first and second derivatives of the inertia
    inertia = np.array(inertia)
    first_derivative = np.diff(inertia)
    second_derivative = np.diff(first_derivative)

    # Find the "elbow" point where the second derivative changes from negative to positive
    elbow_point = np.where(second_derivative > 0)[0][0] + 2  # Add 2 to account for the two differences calculated

    optimal_k = elbow_point + 1  # You can determine this value visually or programmatically

    k_value2 = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    k_value2.fit(feature.reshape(-1, 1))

    # Assign clusters to data points
    labels = kmeans.predict(feature.reshape(-1, 1))

    return k_value2

In [ ]:
def kMeans3(feature):
    # Calculate the sum of squared distances (inertia) for different values of k
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(feature.reshape(-1, 1))
        inertia.append(kmeans.inertia_)

    # Calculate the first and second derivatives of the inertia
    inertia = np.array(inertia)
    first_derivative = np.diff(inertia)
    second_derivative = np.diff(first_derivative)

    # Find the "elbow" point where the second derivative changes from negative to positive
    elbow_point = np.where(second_derivative > 0)[0][0] + 2  # Add 2 to account for the two differences calculated

    optimal_k = elbow_point + 1  # You can determine this value visually or programmatically

    k_value3 = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    k_value3.fit(feature.reshape(-1, 1))

    # Assign clusters to data points
    labels = kmeans.predict(feature.reshape(-1, 1))
    return k_value3

In [ ]:
def kMeans4(feature):
    # Calculate the sum of squared distances (inertia) for different values of k
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(feature.reshape(-1, 1))
        inertia.append(kmeans.inertia_)

    # Calculate the first and second derivatives of the inertia
    inertia = np.array(inertia)
    first_derivative = np.diff(inertia)
    second_derivative = np.diff(first_derivative)

    # Find the "elbow" point where the second derivative changes from negative to positive
    elbow_point = np.where(second_derivative > 0)[0][0] + 2  # Add 2 to account for the two differences calculated

    optimal_k = elbow_point + 1  # You can determine this value visually or programmatically

    k_value4 = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    k_value4.fit(feature.reshape(-1, 1))

    # Assign clusters to data points
    labels = kmeans.predict(feature.reshape(-1, 1))
    return k_value4

In [ ]:
def kMeans5(feature):
    # Calculate the sum of squared distances (inertia) for different values of k
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(feature.reshape(-1, 1))
        inertia.append(kmeans.inertia_)

    # Calculate the first and second derivatives of the inertia
    inertia = np.array(inertia)
    first_derivative = np.diff(inertia)
    second_derivative = np.diff(first_derivative)

    # Find the "elbow" point where the second derivative changes from negative to positive
    elbow_point = np.where(second_derivative > 0)[0][0] + 2  # Add 2 to account for the two differences calculated

    optimal_k = elbow_point + 1  # You can determine this value visually or programmatically

    k_value5 = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    k_value5.fit(feature.reshape(-1, 1))

    # Assign clusters to data points
    labels = kmeans.predict(feature.reshape(-1, 1))
    return k_value5

In [ ]:
def kMeans6(feature):
    # Calculate the sum of squared distances (inertia) for different values of k
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(feature.reshape(-1, 1))
        inertia.append(kmeans.inertia_)

    # Calculate the first and second derivatives of the inertia
    inertia = np.array(inertia)
    first_derivative = np.diff(inertia)
    second_derivative = np.diff(first_derivative)

    # Find the "elbow" point where the second derivative changes from negative to positive
    elbow_point = np.where(second_derivative > 0)[0][0] + 2  # Add 2 to account for the two differences calculated

    optimal_k = elbow_point + 1  # You can determine this value visually or programmatically

    k_value6 = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    k_value6.fit(feature.reshape(-1, 1))

    # Assign clusters to data points
    labels = kmeans.predict(feature.reshape(-1, 1))
    return k_value6

In [ ]:
k_value1 = kMeans1(np.asarray(feature_values))
k_value2 = kMeans2(np.asarray(density_values))
k_value3 = kMeans3(np.asarray(orientation_values))
k_value4 = kMeans4(np.asarray(ridge_density))
k_value5 = kMeans5(np.asarray(average_beadsize))
k_value6 = kMeans6(np.asarray(average_IR_distance))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
def ID(image_path):
    identification_array=np.zeros(6)
    output = []

    image1 = cv2.imread(image_path)
    image2 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


    hessian_img1 = hessian(image1)

    hessian_img2 = hessian(image2)
    new_image1 = (hessian_img1 * 255).astype(np.uint8)
    new_image2 = (hessian_img2 * 255).astype(np.uint8)
    #new_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    feature_value1 = calculate_bead_arrangement_distance(image_path)
    output.append(feature_value1)

    feature_value2 = round(extract_bead_density(new_image2)*10e5)
    output.append(feature_value2)

    feature_value3 = calculate_ridge_orientation(image_path, analyze_left_side=True, num_orientations=16)
    output.append(feature_value3)

    #imagerg =  cv2.cvtColor(image_path, cv2.COLOR_BGR2GRAY)
    feature_value4 = round(calculate_ridge_density(new_image1)*10e4)
    output.append(feature_value4)

    feature_value5 = round(average_bead_size(new_image1))*10e-2
    output.append(feature_value5)

    feature_value6 = calculate_average_inter_ridge_distance(image_path)
    output.append(feature_value6)

    output_array=np.array(output)

    input1=np.array(output_array[0])
    input1=input1.reshape(-1,1)
    value1=k_value1.predict(input1)
    identification_array[0]=value1+1

    input2=np.array(output_array[1])
    input2=input2.reshape(-1,1)
    value2=k_value2.predict(input2)
    identification_array[1]=value2+1

    input3=np.array(output_array[2])
    input3=input3.reshape(-1,1)
    value3=k_value3.predict(input3)
    identification_array[2]=value3+1

    input4=np.array(output_array[3])
    input4=input4.reshape(-1,1)
    value4=k_value4.predict(input4)
    identification_array[3]=value4+1

    input5=np.array(output_array[4])
    input5=input5.reshape(-1,1)
    value5=k_value5.predict(input5)
    identification_array[4]=value5+1

    input6=np.array(output_array[5])
    input6=input6.reshape(-1,1)
    value6=k_value6.predict(input6)
    identification_array[5]=value6+1

    ans=0
    for i in range(0,6):
        ans=ans*10+identification_array[i]


    return int(ans)

In [ ]:
image_directory = "/content/iota-h"
IDs = []
for filename in os.listdir(image_directory):


    image_path = os.path.join(image_directory, filename)
    ID1 = ID(image_path)
    IDs.append(ID1)

/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to in

In [ ]:
print(IDs)

[323221, 323132, 123122, 121223, 221223, 121123, 221223, 221223, 121123, 121113, 321221, 122212]


In [ ]:
image_path = "/content/new_img22.jpg"
ID1 = ID(image_path)


/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)


In [ ]:
if ID1 in IDs:
  print('Cattle already present in the dataset')

else:
  print('New Cattle')

Cattle already present in the dataset
